# PyPower

In [ ]:
from pypower.runpf import runpf
import pypower

In [ ]:
import sys
sys.path.append("../../data/case9/")

In [ ]:
from case9 import case9

In [ ]:
case = case9()

![]("./9BusSystem.jpg")

In [ ]:
ppopt = pypower.ppoption.ppoption(VERBOSE=0)

### Initialization

In [ ]:
# -*- coding: utf-8 -*-
import time
import helics as h
from math import pi

initstring = "-f 2 --name=mainbroker"
fedinitstring = "--broker=mainbroker --federates=1"
deltat = 0.01

helicsversion = h.helicsGetVersion()

print("Helics version = {}".format(helicsversion))

### Create the broker

In [ ]:

print("Creating Broker")
broker = h.helicsCreateBroker("zmq", "", initstring)
print("Created Broker")

print("Checking if Broker is connected")
isconnected = h.helicsBrokerIsConnected(broker)
print("Checked if Broker is connected")

if isconnected == 1:
    print("Broker created and connected")

### Create the federate info object

In [ ]:
# Create Federate Info object that describes the federate properties #
fedinfo = h.helicsCreateFederateInfo()

# Set Federate name #
h.helicsFederateInfoSetCoreName(fedinfo, "PyPowerFederate")

# Set core type from string #
h.helicsFederateInfoSetCoreTypeFromString(fedinfo, "zmq")

# Federate init string #
h.helicsFederateInfoSetCoreInitString(fedinfo, fedinitstring)

# Set the message interval (timedelta) for federate. Note th#
# HELICS minimum message time interval is 1 ns and by default
# it uses a time delta of 1 second. What is provided to the
# setTimedelta routine is a multiplier for the default timedelta.

# Set one second message interval #
h.helicsFederateInfoSetTimeProperty(fedinfo, h.helics_property_time_delta, deltat)

### Create a value federate

In [ ]:
# Create value federate #
vfed = h.helicsCreateValueFederate("PyPowerFederate", fedinfo)
print("Value federate created")

# Register the publication #
pub = h.helicsFederateRegisterGlobalTypePublication(vfed, "voltage", "complex", "")
sub = h.helicsFederateRegisterSubscription(vfed, "load", "")

### Enter execution

In [ ]:
# Enter execution mode #
h.helicsFederateEnterExecutingMode(vfed)
print("Entering execution mode")

### Start simulation

In [ ]:
import numpy as np

In [ ]:
def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return x, y

In [ ]:

granted_time = -1

for request_time in range(1, 300):
    
    while granted_time < request_time:
        granted_time = h.helicsFederateRequestTime(vfed, request_time)
    
    Pg, Qg = h.helicsInputGetComplex(sub)
    print("Active power at time {}: {} ".format(request_time, Pg))
    print("Reactive power at time {}: {} ".format(request_time, Qg))

    case["bus"][4][3:5] = Pg, Qg
    r, s = runpf(case, fname="/dev/null", ppopt=ppopt)
    voltage, angle = r["bus"][4][7:9]
    
    print("Source voltage mag at time {}: {} ".format(request_time, voltage))
    print("Source voltage ang at time {}: {} ".format(request_time, angle))

    h.helicsPublicationPublishComplex(pub, voltage, angle)
    
    print()


h.helicsFederateFinalize(vfed)
print("Federate finalized")

while h.helicsBrokerIsConnected(broker):
    time.sleep(1)

h.helicsFederateFree(vfed)
h.helicsCloseLibrary()

print("Broker disconnected")